##  Importer les bibliothèques 

In [130]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
from mlflow import MlflowClient
import warnings

In [131]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
print(train_df.columns)
print(test_df.columns)

warnings.filterwarnings('ignore')

# Vérifier la présence de la colonne 'Embarked'
if 'Embarked' in train_df.columns:
    # Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente (si présente)
    train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans train_df")

if 'Embarked' in test_df.columns:
    test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)
else:
    print("La colonne 'Embarked' n'est pas présente dans test_df")

# Remplir les valeurs manquantes pour 'Age' avec la médiane
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# Remplir les valeurs manquantes pour 'Fare' dans le jeu de test avec la médiane
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Supprimer la colonne 'Cabin' car elle contient trop de valeurs manquantes
train_df.drop('Cabin', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

# Créer une nouvelle feature 'FamilySize' à partir de 'SibSp' et 'Parch'
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Créer une nouvelle feature 'IsAlone' à partir de 'FamilySize'
train_df['IsAlone'] = 1
train_df.loc[train_df['FamilySize'] > 1, 'IsAlone'] = 0
test_df['IsAlone'] = 1
test_df.loc[test_df['FamilySize'] > 1, 'IsAlone'] = 0

# Supprimer les colonnes inutiles
train_df = train_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)

# Convertir la feature 'Sex' en numérique
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

# Convertir la feature 'Embarked' en numérique en utilisant le one-hot encoding (à voir les tech)
train_df = pd.get_dummies(train_df, columns=['Embarked'])
test_df = pd.get_dummies(test_df, columns=['Embarked'])

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [132]:
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##  Charger le modèle depuis MLflow

In [133]:
mlflow_client = MlflowClient()
model_name = "TitanicModel2"
filter_string = f"name='{model_name}'"
results = client.search_registered_models(filter_string=filter_string)
print(results)
print("-" * 80)
for res in results:
    for mv in res.latest_versions:
        run_id=mv.run_id
        print(f"name={mv.name}; run_id={mv.run_id}; version={mv.version}")




[<RegisteredModel: aliases={}, creation_timestamp=1719820693892, description=None, last_updated_timestamp=1720017205620, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720017205620, current_stage='None', description=None, last_updated_timestamp=1720017205620, name='TitanicModel2', run_id='dfeff3f8b99a4c86a2069522950553b5', run_link=None, source='runs:/dfeff3f8b99a4c86a2069522950553b5/model', status='READY', status_message=None, tags={}, user_id=None, version=12>], name='TitanicModel2', tags={}>]
--------------------------------------------------------------------------------
name=TitanicModel2; run_id=dfeff3f8b99a4c86a2069522950553b5; version=12


In [134]:
registered_versions = mlflow_client.search_registered_models(filter_string=filter_string,
    order_by=(["last_updated_timestamp DESC"]))
print(registered_versions)
last_model_uri=registered_versions[0].latest_versions[-1].source    
print(f"last_model_uri : {last_model_uri}")    # Load the appropriate model identified above    


[<RegisteredModel: aliases={}, creation_timestamp=1719820693892, description=None, last_updated_timestamp=1720017205620, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720017205620, current_stage='None', description=None, last_updated_timestamp=1720017205620, name='TitanicModel2', run_id='dfeff3f8b99a4c86a2069522950553b5', run_link=None, source='runs:/dfeff3f8b99a4c86a2069522950553b5/model', status='READY', status_message=None, tags={}, user_id=None, version=12>], name='TitanicModel2', tags={}>]
last_model_uri : runs:/dfeff3f8b99a4c86a2069522950553b5/model


In [135]:
# Charger le modèle depuis MLflow
loaded_model = mlflow.sklearn.load_model(last_model_uri)

## Préparer les nouvelles données

## Appliquer le modèle pour faire des prédictions

In [140]:
# Appliquer le modèle pour faire des prédictions
predictions = loaded_model.predict(X)

# Générer le fichier submission.csv
submission_pred = pd.DataFrame({'PassengerId': new_data['PassengerId'], 'Survived': predictions})
submission_pred.to_csv('/submission_pred.csv', index=False)

print("Les prédictions ont été enregistrées dans submission_pred.csv")


ValueError: array length 891 does not match index length 89